<a href="https://colab.research.google.com/github/RyanChen12035/w266-NLP/blob/main/w266_final_project_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
1. Use GPT2-medium, Decoder based, pretrained with EN text and has 24 layers of transformer layers. On top of it is a projection layer with shape(768, 50257) and softmax layer
2. It was pretrained by EN text, so basically it can not handle EN to CN task without fine-tuning
3. Need a token that incorperate EN and CN, the embedding size and dictionary size are (768, 119,547)
4. Fine-tune, may need prompt + EN,CN pair. Detailed training process needs to be further clearfy.
5. The FFN consists of two linear transformations with weights of shapes (768, 3072) and (3072, 768)
6. Only need a look-ahead mask before the first layer of transformer. For following layers, input would be pass down one by one without seeing future information.
7. No CLS token in decoder based model, so after 10 runs of decoding process (assuming we have 10 input length, pass one token and previous input for
   next layer each time), we would have (batch, sequence, embedding dimensions) as the shape of each transformer layer. We can take mean pooling or max pooling
   to have the representation of each layers of transformer.
8. The shape of representation would be (1,768) and we can put it into the comparison of neurons in FFN.

"""

In [20]:
import os
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pprint import pprint
from IPython.display import clear_output
from transformers import BertTokenizer, GPT2LMHeadModel
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout, Conv1D, GlobalMaxPooling1D, Concatenate, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from transformers import logging
logging.set_verbosity_error()
import sklearn as sk
import os
from nltk.data import find
import matplotlib.pyplot as plt
import re

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-a7be4f05-fc57-d38b-88c6-7750b857c94d)


In [4]:
#check out the data source we have
tmp_builder = tfds.builder("wmt19_translate/zh-en")
pprint(tmp_builder.subsets)

{Split('train'): ['newscommentary_v14',
                  'wikititles_v1',
                  'uncorpus_v1',
                  'casia2015',
                  'casict2011',
                  'casict2015',
                  'datum2015',
                  'datum2017',
                  'neu2017'],
 Split('validation'): ['newstest2018']}


In [6]:
#download data by tfds.builder
config = tfds.translate.wmt.WmtConfig(
  version=tfds.core.Version('0.0.3'),
  language_pair=("zh", "en"),
  subsets={
    tfds.Split.TRAIN: ["newscommentary_v14"]
  }
)
builder = tfds.builder("wmt_translate", config=config)

# Download and prepare the dataset
builder.download_and_prepare()

# Set builder to dataset (data pipeline type), split it into training, validation, testing
examples = builder.as_dataset(
    split=['train[:20%]', 'train[20%:21%]', 'train[21%:]'],
    as_supervised=True
)

clear_output()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/wmt_translate/zh-en/0.0.3.incompleteH8QB5H/wmt_translate-train.tfrecord*..…

Dataset wmt_translate downloaded and prepared to /root/tensorflow_datasets/wmt_translate/zh-en/0.0.3. Subsequent calls will reuse this data.


In [10]:
#leave the testing examples this time.
train_examples, val_examples, test_examples = examples
print(train_examples)
print(val_examples)

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>
<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>


In [17]:
num_samples = 10

for en_t, zh_t in train_examples.take(num_samples):
  en = en_t.numpy().decode("utf-8")
  zh = zh_t.numpy().decode("utf-8")

  print(en)
  print(zh)
  print('-' * 10)

b'The fear is real and visceral, and politicians ignore it at their peril.'
The fear is real and visceral, and politicians ignore it at their peril.
这种恐惧是真实而内在的。 忽视它的政治家们前途堪忧。
----------
b'In fact, the German political landscape needs nothing more than a truly liberal party, in the US sense of the word \xe2\x80\x9cliberal\xe2\x80\x9d \xe2\x80\x93 a champion of the cause of individual freedom.'
In fact, the German political landscape needs nothing more than a truly liberal party, in the US sense of the word “liberal” – a champion of the cause of individual freedom.
事实上，德国政治局势需要的不过是一个符合美国所谓“自由”定义的真正的自由党派，也就是个人自由事业的倡导者。
----------
b'Shifting to renewable-energy sources will require enormous effort and major infrastructure investment.'
Shifting to renewable-energy sources will require enormous effort and major infrastructure investment.
必须付出巨大的努力和基础设施投资才能完成向可再生能源的过渡。
----------
b'In this sense, it is critical to recognize the fundamental difference between \xe2\x80\x9curban villages\xe2\x8

In [21]:
# Initialize a tokenizer for English and Chinese
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# Initialize the GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [18]:
# Prepare the Tokenizer, it must be able to handle both EN and ZH
# ZH as input and EN as target output.
# Define the Data Processing Function
# 1. encode to utf-8
# 2. set up max_length andadd padding
# Apply the Function to the Dataset


# Create Data Loaders: Prepare data loaders for training, validation, and testing datasets.



def encode_examples(zh_texts, en_texts, max_length=512):
    input_ids = []
    labels = []

    for zh, en in zip(zh_texts, en_texts):
        prompt = "Translate from Chinese to English: "
        zh_input = prompt + zh.numpy().decode('utf-8')

        # Tokenize the combined prompt and Chinese text
        inputs = tokenizer.encode(zh_input, return_tensors="tf", padding="max_length", max_length=max_length, truncation=True)

        # Tokenize the English text for labels (targets)
        target = tokenizer.encode(en.numpy().decode('utf-8'), return_tensors="tf", padding="max_length", max_length=max_length, truncation=True)

        input_ids.append(inputs)
        labels.append(target)

    return input_ids, labels

def map_function(zh, en):
    return tf.py_function(encode_examples, inp=[zh, en], Tout=[tf.int32, tf.int32])

# Use TensorFlow's Dataset API to apply the function
# customized encoding method.
train_dataset = train_examples.map(map_function)
val_dataset = val_examples.map(map_function)
test_dataset = test_examples.map(map_function)


In [19]:
batch_size = 16  # Adjust based on your GPU memory


train_dataset = train_dataset.shuffle(10000).batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

AttributeError: ignored

In [1]:
# validation?
# Do we need masks?
# The purpose of fine tuning in this case.
# We fine tune BERT but as for GPT we can apply prompt to ask the model to do the specific task.

#https://medium.com/@kapildevkhatik2/mastering-gpt-3-a-comprehensive-guide-to-fine-tuning-with-openai-complete-with-examples-e28515c22d92
#https://www.linkedin.com/pulse/fine-tuning-gpt-35-turbo-translation-terence-lewis/
#https://clashpanda.com/how-to-fine-tune-chat-gpt-for-translation-tasks/#google_vignette